In [2]:
pip install pandas 

  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl (11.0 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Import Libraries

In [3]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns',None)
pd.set_option('display.width',120)

Load Data

In [4]:
train = pd.read_csv('/Users/battih/Desktop/Personal/demand-forecasting-walmart/data/raw/train.csv')
features = pd.read_csv('/Users/battih/Desktop/Personal/demand-forecasting-walmart/data/raw/features.csv')
stores = pd.read_csv('/Users/battih/Desktop/Personal/demand-forecasting-walmart/data/raw/stores.csv')

train.head(), features.head(), stores.head()

(   Store  Dept        Date  Weekly_Sales  IsHoliday
 0      1     1  2010-02-05      24924.50      False
 1      1     1  2010-02-12      46039.49       True
 2      1     1  2010-02-19      41595.55      False
 3      1     1  2010-02-26      19403.54      False
 4      1     1  2010-03-05      21827.90      False,
    Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2  MarkDown3  MarkDown4  MarkDown5         CPI  \
 0      1  2010-02-05        42.31       2.572        NaN        NaN        NaN        NaN        NaN  211.096358   
 1      1  2010-02-12        38.51       2.548        NaN        NaN        NaN        NaN        NaN  211.242170   
 2      1  2010-02-19        39.93       2.514        NaN        NaN        NaN        NaN        NaN  211.289143   
 3      1  2010-02-26        46.63       2.561        NaN        NaN        NaN        NaN        NaN  211.319643   
 4      1  2010-03-05        46.50       2.625        NaN        NaN        NaN        NaN      

Inspect Shapes and Columns

In [6]:
print('Train Shape: ', train.shape)
print('Features shape: ', features.shape)
print('Stores shape: ', stores.shape)

Train Shape:  (421570, 5)
Features shape:  (8190, 12)
Stores shape:  (45, 3)


In [8]:
print('Train columns: ', train.columns.to_list())
print('\nFeatures columns: ', features.columns.to_list())
print('\nStores columns: ', stores.columns.to_list())

Train columns:  ['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday']

Features columns:  ['Store', 'Date', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'IsHoliday']

Stores columns:  ['Store', 'Type', 'Size']


Target Variable

In [9]:
train['Weekly_Sales'].describe()

count    421570.000000
mean      15981.258123
std       22711.183519
min       -4988.940000
25%        2079.650000
50%        7612.030000
75%       20205.852500
max      693099.360000
Name: Weekly_Sales, dtype: float64

Converting Date Columns

In [13]:
train['Date'] = pd.to_datetime(train['Date'])
features['Date'] = pd.to_datetime(features['Date'])

In [14]:
train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [15]:
features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [16]:
stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


Merge datasets

In [18]:
df = train.merge(
    features,
    on=['Store','Date'],
    how='left').merge(
        stores,
        on='Store',
        how='left'
        )

df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,A,151315
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True,A,151315
2,1,1,2010-02-19,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False,A,151315
3,1,1,2010-02-26,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False,A,151315
4,1,1,2010-03-05,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False,A,151315


In [19]:
df[['Dept', 'Store']].drop_duplicates().shape

(3331, 2)

In [20]:
sample_series = df[(df['Store']==1) & (df['Dept']==1)]
sample_series.sort_values('Date').head(10)

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,A,151315
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True,A,151315
2,1,1,2010-02-19,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False,A,151315
3,1,1,2010-02-26,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False,A,151315
4,1,1,2010-03-05,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False,A,151315
5,1,1,2010-03-12,21043.39,False,57.79,2.667,NaN,NaN,NaN,NaN,NaN,211.380643,8.106,False,A,151315
6,1,1,2010-03-19,22136.64,False,54.58,2.720,NaN,NaN,NaN,NaN,NaN,211.215635,8.106,False,A,151315
7,1,1,2010-03-26,26229.21,False,51.45,2.732,NaN,NaN,NaN,NaN,NaN,211.018042,8.106,False,A,151315
8,1,1,2010-04-02,57258.43,False,62.27,2.719,NaN,NaN,NaN,NaN,NaN,210.820450,7.808,False,A,151315
9,1,1,2010-04-09,42960.91,False,65.86,2.770,NaN,NaN,NaN,NaN,NaN,210.622857,7.808,False,A,151315


Missing values

In [21]:
df.isna().sum()

Store                0
Dept                 0
Date                 0
Weekly_Sales         0
IsHoliday_x          0
Temperature          0
Fuel_Price           0
MarkDown1       270889
MarkDown2       310322
MarkDown3       284479
MarkDown4       286603
MarkDown5       270138
CPI                  0
Unemployment         0
IsHoliday_y          0
Type                 0
Size                 0
dtype: int64

In [22]:
df.isna().mean().sort_values(ascending=False)

MarkDown2       0.736110
MarkDown4       0.679847
MarkDown3       0.674808
MarkDown1       0.642572
MarkDown5       0.640790
Store           0.000000
IsHoliday_x     0.000000
Dept            0.000000
Date            0.000000
Temperature     0.000000
Fuel_Price      0.000000
Weekly_Sales    0.000000
CPI             0.000000
Unemployment    0.000000
IsHoliday_y     0.000000
Type            0.000000
Size            0.000000
dtype: float64

In [23]:
df.to_csv('/Users/battih/Desktop/Personal/demand-forecasting-walmart/data/processed/base_dataset.csv',index=False)